In [23]:
import imutils
import numpy as np
import cv2

In [24]:
from tensorflow.keras.models import load_model
model1 = load_model('F:/Project_Mask_NoMask/model1.h5')


In [25]:
!curl -O https://raw.githubusercontent.com/opencv/opencv/master/samples/dnn/face_detector/deploy.prototxt
!curl -O https://raw.githubusercontent.com/opencv/opencv_3rdparty/dnn_samples_face_detector_20170830/res10_300x300_ssd_iter_140000.caffemodel

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0 28104    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 28104  100 28104    0     0  46411      0 --:--:-- --:--:-- --:--:-- 46529
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  2 10.1M    2  216k    0     0   268k      0  0:00:38 --:--:--  0:00:38  268k
 25 10.1M   25 2630k    0     0  1406k      0  0:00:07  0:00:01  0:00:06 1407k
 51 10.1M   51 5382k    0     0  1911k      0  0:00:05  0:00:02  0:00:03 1911k
 74 10.1M   74 7718k    0     0  2026k      0  0:00:05  0:00:03  0:00:02 2027k
100 10.1M  100 10.1M    0     0  2209k      0  0:0

In [26]:
print("[INFO] loading model...")
prototxt = 'deploy.prototxt'
model = 'res10_300x300_ssd_iter_140000.caffemodel'
net = cv2.dnn.readNetFromCaffe(prototxt, model)

[INFO] loading model...


In [33]:
import cv2

 

cap = cv2.VideoCapture(0)

while True:

    ret, frame = cap.read()

    if ret:

        # Excute frame here
        #frame = cv2.imread(frame)
        
        frame = imutils.resize(frame, width=400)
        (h, w) = frame.shape[:2]
        blob = cv2.dnn.blobFromImage(cv2.resize(frame, (300, 300)), 1.0, (300, 300), (104.0, 177.0, 123.0))
        net.setInput(blob)
        detections = net.forward()
        for i in range(0, detections.shape[2]):
            confidence = detections[0, 0, i, 2]
            if confidence > 0.5:
                box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
                (startX, startY, endX, endY) = box.astype("int")
                text = "{:.2f}%".format(confidence * 100)
                y = startY - 10 if startY - 10 > 10 else startY + 10
                cv2.rectangle(frame, (startX, startY), (endX, endY), (0, 0, 255), 2)
                minX, maxX = min(startX, endX), max(startX, endX)
                minY, maxY = min(startY, endY), max(startY, endY)
                face = frame[minY:maxY, minX:maxX].copy()
                img = cv2.cvtColor(face,cv2.COLOR_BGR2GRAY)
                img = cv2.resize(img,(128,128))
                img = np.array([img/255.])
                y_hat = model1.predict(img)
                for i in range(y_hat.shape[1]):
                    max_value=np.max(y_hat)
                    if y_hat[0][i]==max_value:
                        y_hat[0][i]=1
                    else:
                        y_hat[0][i]=0
                predic = np.argmax(y_hat, axis=1)
                if predic == 0:
                    cv2.putText(frame,"0", (startX, y),cv2.FONT_HERSHEY_SIMPLEX, 0.45, (0, 0, 255), 2)
                if predic ==1:
                    cv2.putText(frame,"1", (startX, y),cv2.FONT_HERSHEY_SIMPLEX, 0.45, (0, 0, 255), 2)
                else:
                    cv2.putText(frame,"2", (startX, y),cv2.FONT_HERSHEY_SIMPLEX, 0.45, (0, 0, 255), 2)
        
        cv2.imshow("frame", frame)
        
        key = cv2.waitKey(1)

        if key == 27:

            break

 

cap.release()

cv2.destroyAllWindows()

1/1 [==============================] - 0s 44ms/step


In [28]:
import os
import matplotlib.image as mpimg
import glob
import shutil
import cv2 
from PIL import Image
import PIL
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import random 
from sklearn.model_selection import train_test_split


import warnings
# Ignore waring
warnings.filterwarnings('ignore')

In [29]:
name_list = glob.glob("/content/happy/*")
X=[]
name_label=[]
for name in name_list:
  name_label.append(0)

  img =cv2.imread(name)
  img = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
  img = cv2.resize(img,(50,50))

  X.append((img))
len(X)

0

In [30]:
name_list = glob.glob("/content/sad/*")

for name in name_list:
  name_label.append(1)

  img = cv2.imread(name)
  img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
  img = cv2.resize(img, (50,50))

  X.append((img))
len(X)

0

In [31]:
X = []
y = []
for i, path in enumerate(mask_path):

  for name in glob.glob(train_path + path):
    img = cv2.imread(name)
    img = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    #img = img.convert('RGB')
    img = cv2.resize(img,(128,128))
    X.append((img))
    y.append(i)
len(X)

NameError: name 'mask_path' is not defined

In [ ]:
X = np.array(X)/255.
y = np.array(y)

In [ ]:
X[0].shape
print(X.shape)
print(y.shape)
print(y)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=2)

In [ ]:
print(X.shape, X_train.shape, X_test.shape)

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import *
from tensorflow.keras.models import Model
from tensorflow.keras import backend as K 

In [ ]:
inp = Input(shape=(128,128,1))

x = Conv2D(filters=32 ,kernel_size=3, activation='relu')(inp)
x = MaxPooling2D(pool_size= (2,2))(x)
x = Dropout(0.25)(x)

x = Conv2D(filters=64 ,kernel_size=3, activation='relu')(inp)
x = MaxPooling2D(pool_size= (2,2))(x)
x = Dropout(0.25)(x)

x = Conv2D(filters=128 ,kernel_size=3, activation='relu')(inp)
x = MaxPooling2D(pool_size= (2,2))(x)
x = Dropout(0.25)(x)

x = Conv2D(filters=128 ,kernel_size=3, activation='relu')(inp)
x = MaxPooling2D(pool_size= (2,2))(x)
x = Dropout(0.25)(x)



x = Flatten()(x)
x = Dense(units = 128,activation='relu')(x)
x = Dense(units = 64,activation='relu')(x)
x = Dense(units = 2)(x)

cnn = Model(inputs = inp , outputs = x )


img1 = Input(shape=(128,128,1))
img2 = Input(shape=(128,128,1))

f1 = cnn(img1)
f2 = cnn(img2)

d = K.sqrt(K.sum(K.square(f1-f2),axis =1 ,keepdims=True))

model = Model(inputs = (img1,img2),outputs = d)
model.summary()
cnn.summary()

def loss(y_true,y_pred):
    proba = K.exp(-K.square(y_pred))
    return -K.mean(y_true*K.log(proba)+(1-y_true)*K.log(1-proba))

def loss1(y_true,y_pred):
    return K.mean(y_true*K.square(y_pred)+(1-y_true)*K.square(K.maximum(1.0-y_pred,0)))

model.compile(optimizer='adam',loss = loss1)


In [ ]:
import numpy as np 
from matplotlib import pyplot as plt
def generator(X,y,k=8):
    unique_labels = np.unique(y)
    while True:
        X1 = []
        X2 = []
        y_batch = []
        for label in unique_labels :
            label_idx = np.where(y==label)[0]
            other_labels =set(unique_labels)-{label}
            for i in range(k):
                i1 = np.random.choice(label_idx)
                i2 = np.random.choice(label_idx)
                # i1 must be different from i2
                
                while i1 == i2 :
                    #i1 = np.random.choice(label_idx)
                    i2 = np.random.choice(label_idx)
                    
                # create positive example 
                X1.append(X[i1][:,:,None])
                X2.append(X[i2][:,:,None])
                y_batch.append(1.0)
                
                # create negative example
                i1 = np.random.choice(label_idx)
                my_label = np.random.choice(list(other_labels))
                i2 = np.random.choice(list(np.where(y == my_label)[0]))
                X1.append(X[i1][:,:,None])
                X2.append(X[i2][:,:,None])
                y_batch.append(0.0)
        
        yield [np.array(X1)/255.,np.array(X2)/255.],np.array(y_batch) 
            
                
           

In [ ]:
history = model.fit(generator(X_train,y_train,k=16),
                    steps_per_epoch = 5,
                    epochs = 10,
                    validation_data = generator(X_test,y_test,k=8),
                    validation_steps= 5)

In [ ]:
loss, accuracy = model.evaluate(X_test_scaled, Y_test)
print('Test Accuracy =', accuracy)

In [ ]:
h = history

# plot the loss value
plt.plot(h.history['loss'], label='train loss')
plt.plot(h.history['val_loss'], label='validation loss')
plt.legend()
plt.show()

# plot the accuracy value
plt.plot(h.history['acc'], label='train accuracy')
plt.plot(h.history['val_acc'], label='validation accuracy')
plt.legend()
plt.show()

In [ ]:
h = history

# plot the loss value
plt.plot(h.history['loss'], label='train loss')
plt.plot(h.history['val_loss'], label='validation loss')
plt.legend()
plt.show()

# plot the accuracy value
plt.plot(h.history['acc'], label='train accuracy')
plt.plot(h.history['val_acc'], label='validation accuracy')
plt.legend()
plt.show()

In [ ]:
input_image_path = input('Path of the image to be predicted: ')

input_image = cv2.imread(input_image_path)

cv2_imshow(input_image)

input_image_resized = cv2.resize(input_image, (128,128))

input_image_scaled = input_image_resized/255

input_image_reshaped = np.reshape(input_image_scaled, [1,128,128,3])

input_prediction = model.predict(input_image_reshaped)

print(input_prediction)


input_pred_label = np.argmax(input_prediction)

print(input_pred_label)


if input_pred_label == 1:

  print('The person in the image is wearing a mask')

else:

  print('The person in the image is not wearing a mask')

In [ ]:
input_image_path = input('Path of the image to be predicted: ')

input_image = cv2.imread(input_image_path)

cv2_imshow(input_image)

input_image_resized = cv2.resize(input_image, (128,128))

input_image_scaled = input_image_resized/255

input_image_reshaped = np.reshape(input_image_scaled, [1,128,128,3])

input_prediction = model.predict(input_image_reshaped)

print(input_prediction)


input_pred_label = np.argmax(input_prediction)

print(input_pred_label)


if input_pred_label == 1:

  print('The person in the image is wearing a mask')

else:

  print('The person in the image is not wearing a mask')